# SRP
## single-responsibility principle

Каждый объект должен иметь одну ответственность и эта ответственность должна быть полностью инкапсулирована в класс. Все его поведения должны быть направлены исключительно на обеспечение этой ответственности.

Предположим, что мы пишем программу для аналитики данных.

На первых этапах разработчики решили реализовать подключение к базе данных.

В качестве базы данных, упрощенно будем использовать словарь:

In [ ]:
DATA = {
    'price': [100, 200, 300],
    'item': ['пиво', 'рыба', 'раки']
}

Реализованное подключение к базе данных:

In [ ]:
class DB:
    def __init__(self, connection_string, data):
        self.__connect = self.__connect_to_db(connection_string)
        self.__data = data
        
    def __connect_to_db(self, connection_string):
        if len(connection_string.split('.')) == 4:
            return connection_string
        else:
            raise ValueError
        
    def select_column(self, name):
        column = self.__data[name]
        return column

In [5]:
db = DB('0.0.0.0')
price = db.select_column('price')
print(price)

[100, 200, 300]


Далее нам нужно реализовать аналитику данных.

Как же это стоит сделать? Начну сначала с плохого примера, который не будет подчиняться `SRP`. Делаю это нарочно, чтобы сразу продемонстрировать от каких проблем может спасти `SRP`. Далее, будет исправленный пример, который подчиняется `SRP`.

В следующем примере программист решил реализовать аналитику прямо внутри класса с базой данных. А почему нет? Это удобно, не нужно заводить дополнительных сущностей, все сразу работает.

In [21]:
class DB:
    def __init__(self, connection_string, data):
        self.__connect = self.__connect_to_db(connection_string)
        self.__data = data
        
    def __connect_to_db(self, connection_string):
        if len(connection_string.split('.')) == 4:
            return connection_string
        else:
            raise ValueError
        
    def select_column(self, name):
        column = self.__data[name]
        return column
    
    def mean_in_column(self, column_name):
        column = self.select_column(column_name)
        # Обратите внимание, я считаю среднее
        # таким образом не просто так.
        # Это нарочный баг.
        # Баги встречаются везде и это нормально.
        # Хоть здесь он и совсем глупый.
        return sum(column) / 3
    
    def max_in_column(self, column_name):
        column = self.select_column(column_name)
        return max(column)

In [22]:
db = DB('0.0.0.0', DATA)

print(db.mean_in_column('price'))
print(db.max_in_column('price'))

200.0
300


Но что же не так с этим классом?

То, что он содержит сразу несколько "бизнес" логики: И коннект к БД И аналитику.

Прошло какое то время, какому то программисту понадобилось считать аналитику не из базы данных. А напрямую из списка:

In [23]:
ANOTHER_PRICES = [101, 404, 228]

Программист видит код аналитики в `DB`, но он реализован так, что сложно или невозможно переиспользовать его на списке `ANOTHER_PRICES`.

Проще всего взять и втупую сделать копипасту. Можно, конечно, вынести нужную часть в отдельную независимую функцию, но смысл примера от этого сильно не изменится.


Так вот, наш программист берет и создает новый класс, в который отправляет копипасту.

In [37]:
class Analytic:
    def mean(self, distribution):
        return sum(distribution) / 3
    
    def max(self, distribution):
        return max(distribution)

In [38]:
analytics = Analytic()

print(analytic.mean(ANOTHER_PRICES))
print(analytic.max(ANOTHER_PRICES))

244.33333333333334
404


Вдруг! Тестировщик нашел баг (или не тестировщик, а клиент нашей программы). Оказывается мы считаем неправильно среднее:

In [27]:
ANOTHER_DATA = {
    'price': [1,1,1,1],
    'item': ['пиво', 'рыба', 'раки', 'кальмар']
}

In [28]:
db = DB('0.0.0.0', ANOTHER_DATA)

assert db.mean_in_column('price') == 1

AssertionError: 

Мы пойдем исправлять `DB`.

Затем, увидим, что в `Analytic` тоже есть баг.

Пойдем исправлять `Analytic`.

...

Все это стоит ресурсов и можно было этого избежать.

Вместо расширения функционала изначальной базы данных, стоило создать два отдельных класса с единственной ответственностью:

In [40]:
class DB:
    def __init__(self, connection_string, data):
        self.__connect = self.__connect_to_db(connection_string)
        self.__data = data
        
    def __connect_to_db(self, connection_string):
        if len(connection_string.split('.')) == 4:
            return connection_string
        else:
            raise ValueError
        
    def select_column(self, name):
        column = self.__data[name]
        return column


class Analytic:
    def mean(self, distribution):
        # Обратите внимание, я считаю среднее
        # таким образом не просто так.
        # Это нарочный баг.
        # Баги встречаются везде и это нормально.
        # Хоть здесь он и совсем глупый.
        return sum(distribution) / 3
    
    def max(self, distribution):
        return max(distribution)


class AnalyticOnDB:
    def __init__(self, db, analytic):
        self.__db = db
        self.__analytic = analytic
        
    def mean_in_column(self, column_name):
        column = self.__db.select_column(column_name)
        return self.__analytic.mean(column)
    
    def max_in_column(self, column_name):
        column = self.__db.select_column(column_name)
        return self.__analytic.mean(column)

И теперь у нас есть возможность использовать отдельно каждый класс.

In [43]:
db = DB('0.0.0.0', DATA)
print(db.select_column('price'))

analytics = Analytic()

print(analytic.mean(ANOTHER_PRICES))
print(analytic.max(ANOTHER_PRICES))

[100, 200, 300]
244.33333333333334
404


При этом, в той же ситуации с обнаруженным багом:

In [34]:
db = DB('0.0.0.0', ANOTHER_DATA)
analytic = Analytic()

db_analytic = AnalyticOnDB(db, analytic)
assert db_analytic.mean_in_column('price') == 1

AssertionError: 

Мы исправим ошибку в `Analytic.mean` и все зависимые классы будут прекрасно работать:

In [35]:
class Analytic:
    def mean(self, distribution):
        return sum(distribution) / len(distribution)
    
    def max(self, distribution):
        return max(distribution)

In [36]:
db = DB('0.0.0.0', ANOTHER_DATA)
analytic = Analytic()

db_analytic = AnalyticOnDB(db, analytic)
assert db_analytic.mean_in_column('price') == 1

_____
Тем не менее, всегда стоит помнить, что ООП и SOLID не всегда необходимы. Иногда это приводит к сильному оверинженерингу. В частности, через чур руководствуясь этим принципом можно наплодить очень много сущностей, с которыми будет тяжело работать, поддерживать и вообще ориентироваться в проекте.